**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/DQN

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/DQN


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns the agent's next action. The parameter epsilon controls the trade-off between exploration and exploitation. If epsilon = 1, the agent will choose all its actions randomly (high exploration but very poor exploitation since the chosen actions might lead to very small rewards). On the contrary, if epsilon = 0, the agent keeps focusing on the actions that gave high rewards in the past (pure exploitation but null exploration since the agent might not have discovered all the actions that perform well in different contexts). The function allows to choose whether to use the epsilon-greedy strategy (previously described) or to follow a pure exploitation strategy (when boolean train = False).


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw, outputdict={"-pix_fmt":"yuv420p"})

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.


The 2D array position indicates whether the rat is at a given position. All the cells non-occupied by the rat have 0 values whereas the cell currently occupied gets the value 1. A part of the information contained in the array state contains the 5x5 sub-array of the array position, corresponding to the cells that the rat can currently observe. What happens if the rat is currently at the border of the island ? For convenience, the array state still contains the 5x5 sub-array of the array position (previously described) so that the array position must have larger dimensions than the island: 2 supplementary cells in each direction. To code those new cells as not real cells, their value in the array position is -1.

The 2D array board contains the rewards available on each cell: 0.5 for cheese, -1 for poison and 0 for nothing to eat. The 5x5 sub-array of the array board, corresponding to the cells that the rat can currently observe, is also passed in the array state. Thus, for the same reasons than for the array position, the array board has larger dimensions than the island. There is no special encoding for those supplementary cells since the specific coding of the array position suffices to avoid the rat to go there.  

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
          # The agent performs a random action
          action = agent.act(state, train=False)

          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          prev_state = state
          state, reward, game_over = env.act(action)

          # Update the counters
          if reward > 0:
              win = win + reward
          if reward < 0:
              lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 13.0/24.0. Average score (-11.0)
Win/lose count 7.0/13.0. Average score (-8.5)
Win/lose count 10.5/19.0. Average score (-8.5)
Win/lose count 9.5/8.0. Average score (-6.0)
Win/lose count 11.5/9.0. Average score (-4.3)
Win/lose count 5.5/13.0. Average score (-4.833333333333333)
Win/lose count 12.0/22.0. Average score (-5.571428571428571)
Win/lose count 10.0/15.0. Average score (-5.5)
Win/lose count 8.0/10.0. Average score (-5.111111111111111)
Win/lose count 11.0/20.0. Average score (-5.5)
Win/lose count 13.0/18.0. Average score (-5.454545454545454)
Win/lose count 5.5/6.0. Average score (-5.041666666666667)
Win/lose count 7.5/14.0. Average score (-5.153846153846154)
Win/lose count 13.0/20.0. Average score (-5.285714285714286)
Win/lose count 5.0/8.0. Average score (-5.133333333333334)
Win/lose count 11.5/9.0. Average score (-4.65625)
Win/lose count 8.0/10.0. Average score (-4.5)
Win/lose count 8.5/11.0. Average score (-4.388888888888889)
Win/lose count 11.0/15.0. Average sco

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}





1. Let us show the Bellman equation. 

By definition, we have that:

$\begin{align}
Q^\pi(s,a) &= \mathbb{E}_{\{s_t, a_t\} \sim p^\pi}  \bigg[ \sum_{t=0}^{T} \gamma^t r(s_t,a_t) | s_0 = s, a_0 = a \bigg] \\
&= \mathbb{E}_{\{s_t, a_t\} \sim p^\pi}  \bigg[ r(s_0, a_0) + \sum_{t=1}^{T} \gamma^t r(s_t,a_t) | s_0 = s, a_0 = a \bigg] \\
&= r(s,a) + \gamma \mathbb{E}_{\{s_t, a_t\} \sim p^\pi}  \bigg[  \sum_{t=1}^{T} \gamma^{t-1} r(s_t,a_t) | s_0 = s, a_0 = a \bigg] \\
&= r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)} \mathbb{E}_{a'\sim \pi(\cdot|s')} \mathbb{E}_{\{s_t, a_t\} \sim p^\pi}  \bigg[  \sum_{t=1}^{T} \gamma^{t-1} r(s_t,a_t) | s_1 = s', a_1 = a' \bigg] \\
&= r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)} \mathbb{E}_{a'\sim \pi(\cdot|s')} \mathbb{E}_{\{s_t, a_t\} \sim p^\pi}  \bigg[  \sum_{t=1}^{T} \gamma^{t-1} r(s_{t-1},a_{t-1}) | s_0 = s', a_0 = a' \bigg] \\
&= r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)} \mathbb{E}_{a'\sim \pi(\cdot|s')} \mathbb{E}_{\{s_t, a_t\} \sim p^\pi}  \bigg[  \sum_{t=0}^{T-1} \gamma^{t} r(s_{t},a_{t}) | s_0 = s', a_0 = a' \bigg] \\
&= r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)} \mathbb{E}_{a'\sim \pi(\cdot|s')} \big[ Q^\pi(s',a') \big]
\end{align}$


One could notice that the last step is an approximation since the sum goes up to $T-1$ instead of $T$. It would have been exact in an infinite time horizon setting. Thus, this equation is only an approximation in our case since we are in a finite horizon setting. 




2. Let us show the optimal Bellman equation.

Assuming that the optimal policy $\pi^*$ exists, the previous equation holds for that policy:

$Q^{\pi^*}(s,a) = Q^*(s,a) =  r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)} \mathbb{E}_{a'\sim \pi^*(\cdot|s')} \big[ Q^*(s',a') \big]$

Since the optimal policy is deterministic and chooses its action $a'$, being in state $s'$, according to $a' = argmax_{a'} Q^*(s',a')$, we get that:

$\begin{align}
Q^*(s,a) &=  r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)}  \bigg[ Q^*\big[s',argmax_{a'} Q^*(s',a')\big] \bigg] \\
&= r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)} max_{a'} Q^*(s',a')
\end{align}$





3. Finally, if we estimate the optimal Q-function $Q^*(s,a)$ by a model $Q(s,a,\theta)$, a good idea to approach the optimal Q-function is to consider the Bellman error, that is the difference between the right-hand and the left-hand sides of the previous equation. This new quantity is equal to 0 if and only if our estimate is equal to the optimal Q-function. Let us write the Bellman error $\mathcal{B}(Q(s,a,\theta))$:

$\begin{align}
\mathcal{B}(Q(s,a,\theta)) &= r(s,a) + \gamma \mathbb{E}_{s'\sim p(\cdot|s,a)} max_{a'} Q(s',a', \theta) - Q(s,a,\theta) \\
&= \mathbb{E}_{s'\sim p(\cdot|s,a)} \big[ r(s,a) + \gamma max_{a'} Q(s',a', \theta) - Q(s,a,\theta) \big]
\end{align}$

To get a differentiable loss function (using Monte-Carlo approximation for instance), a natural choice is then:

$\mathcal{L}(\theta) = \mathbb{E}_{s'\sim p(\cdot|s,a)} | r(s,a) + \gamma max_{a'} Q(s',a', \theta) - Q(s,a,\theta) |^2$




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) > self.max_memory:
            del self.memory[0]  # Keep the maximum number of recent moves

        self.memory.append(m) # Add to replay buffer

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # Arrays to store new batch
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        input_new_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        q_next = np.zeros((self.batch_size, 4))
        actions = np.zeros(self.batch_size, dtype = int)
        rewards = np.zeros(self.batch_size)
        over = []

        # Batch creation
        for i in range(self.batch_size):
            # Sample one transition
            s, n_s, a, r, game_over = self.memory.random_access()

            # Put first state in batch
            input_states[i,:,:,:] = s
            # Put next state in batch
            input_new_states[i,:,:,:] = n_s
            # Put action in batch
            actions[i] = int(a) 
            # Put reward in batch
            rewards[i] = r
            # Same for boolean game_over
            over.append(game_over)

        # First compute current model of Q(s_t, a_t, theta)
        # Indeed, since we use a bootstrapped target (with a maximum) for
        # a pair (s_t, a_t), only the value of Q(s_t, a_t, theta) is concerned,
        # that is one unique action a_t for a given input s_t.
        # For other actions than a_t for the input s_t, we keep the prediction
        # of the current model
        target_q[:,:] = self.model.predict(input_states)

        # Compute the Q-function to maximize
        q_next[:,:] = self.model.predict(input_new_states)

        # Compute bootstrapped target for batched transitions
        for i in range(self.batch_size):
            if game_over:
              target_q[i, actions[i]] = rewards[i] # q_next not defined at s_t+1 if game over
            else:
              target_q[i, actions[i]] = rewards[i] + self.discount * np.max(q_next[i, :])
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(16, activation="relu"))
        model.add(Dense(16, activation="relu"))
        model.add(Dense(4, activation=None))         
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 200, prefix='fc_train')
HTML(display_videos('fc_train190.mp4'))

Epoch 000/200 | Loss 0.0039 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/200 | Loss 0.0080 | Win/lose count 3.5/6.0 (-2.5)
Epoch 002/200 | Loss 0.0091 | Win/lose count 6.0/1.0 (5.0)
Epoch 003/200 | Loss 0.0066 | Win/lose count 0.5/4.0 (-3.5)
Epoch 004/200 | Loss 0.0034 | Win/lose count 9.0/8.0 (1.0)
Epoch 005/200 | Loss 0.0053 | Win/lose count 3.5/3.0 (0.5)
Epoch 006/200 | Loss 0.0035 | Win/lose count 6.0/12.0 (-6.0)
Epoch 007/200 | Loss 0.0118 | Win/lose count 9.5/3.0 (6.5)
Epoch 008/200 | Loss 0.0010 | Win/lose count 2.0/2.0 (0.0)
Epoch 009/200 | Loss 0.0021 | Win/lose count 3.5/5.0 (-1.5)
Epoch 010/200 | Loss 0.0105 | Win/lose count 8.5/11.0 (-2.5)
Epoch 011/200 | Loss 0.0028 | Win/lose count 9.5/2.0 (7.5)
Epoch 012/200 | Loss 0.0040 | Win/lose count 7.0/6.0 (1.0)
Epoch 013/200 | Loss 0.0127 | Win/lose count 7.5/6.0 (1.5)
Epoch 014/200 | Loss 0.0044 | Win/lose count 5.0/1.0 (4.0)
Epoch 015/200 | Loss 0.0014 | Win/lose count 4.0/5.0 (-1.0)
Epoch 016/200 | Loss 0.0029 | Win/lose count 8.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
                
        model = Sequential([
            Conv2D(32, 3, input_shape=(5, 5, self.n_state), activation='relu', padding="same"),
            Conv2D(64, 3, activation='relu', padding="same"),
            #Conv2D(32, 3, activation='relu', padding="same"),
            Flatten(),
            Dense(self.n_action)  # output
        ])
        
        model.summary()        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,200,prefix='cnn_train')
HTML(display_videos('cnn_train190.mp4'))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 6404      
Total params: 25,508
Trainable params: 25,508
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0045 | Win/lose count 6.0/7.0 (-1.0)
Epoch 001/200 | Loss 0.0022 | Win/lose count 8.0/6.0 (2.0)
Epoch 002/200 | Loss 0.0050 | Win/lose count 4.5/3.0 (1.5)
Epoch 003/200 | Loss 0.0181 | Win/lose count 9.0/12.0 (-3.0)
Epoch 004

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [55]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_20 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 6404      
Total params: 25,508
Trainable params: 25,508
Non-trainable params: 0
_________________________________________________________________
Test of the CNN
Win/lose count 8.0/0. Average score (8.0)
Win/lose count 13.5/0. Average score (10.75)
Win/lose count 8.5/0. Average score (10.0)
Win/lose count 2.0/0. Average score (8.0)
Win/lose count 5.0/0. Average score (7.4)
Win/lose count 12

In [56]:
HTML(display_videos('cnn_test10.mp4'))

In [57]:
HTML(display_videos('fc_test10.mp4'))

The CNN agent always performs better than the FC one. This is due to the grid-aspect of the input (images), well adapted to CNN which extract relevant features from this kind of input. When looking at the results of the CNN agent at each epoch, one can notice that it is really good at avoiding poisonous cells (lose = 0) and quite good at eating cheese (with varying performance). The FC agent is not really good at finding cheese but does not eat poison so much.

When looking at the resulting videos, we observe that both algorithms often get stuck in cyclic movements, resulting in a null global movement. It often happens when there is no cheese close to the agents. In other words, there is not enough exploration. The exploitation though is quite good for the CNN agent.

This lack of exploration is even more significant at lower temperature. Both agents struggle to eat anything in that case.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='', factor = 0.99):
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Decreasing epsilon-greedy exploration
        agent.set_epsilon(agent.epsilon * factor)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.2f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw, outputdict={"-pix_fmt":"yuv420p"})

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train= False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        # Incorporate malus of already visited position
        reward = 0
        if train:
          reward = -self.malus_position[self.x, self.y] # No malus if newly visited
        self.malus_position[self.x, self.y] = 0.2       # Future malus for current position
                    # Tuned value for threshold between 
                    # 1. always move even if eating poison
                    # 2. don't move and avoid poison

        reward = reward + self.board[self.x, self.y]
        
        self.t = self.t + 1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

# You will have to change n_state to 3 because you will use one more layer!

In [45]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.35, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 200, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore190.mp4'))

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 5, 5, 32)          896       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_17 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 6404      
Total params: 25,796
Trainable params: 25,796
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0080 | Win/lose count 7.5/34.2 (-26.70)
Epoch 001/200 | Loss 0.0087 | Win/lose count 17.2/35.6 (-18.40)
Epoch 002/200 | Loss 0.0073 | Win/lose count 8.3/41.4 (-33.10)
Epoch 003/200 | Loss 0.0051 | Win/lose count 12.1/36.0 (-

In [52]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 23.5/6.0. Average score (17.5)
Win/lose count 16.5/7.0. Average score (13.5)
Win/lose count 21.5/7.0. Average score (13.833333333333334)
Win/lose count 18.0/3.0. Average score (14.125)
Win/lose count 21.5/7.0. Average score (14.2)
Win/lose count 17.0/4.0. Average score (14.0)
Win/lose count 21.0/6.0. Average score (14.142857142857142)
Win/lose count 23.0/5.0. Average score (14.625)
Win/lose count 4.5/2.0. Average score (13.277777777777779)
Win/lose count 23.5/2.0. Average score (14.1)
Win/lose count 12.5/4.0. Average score (13.590909090909092)
Win/lose count 23.5/6.0. Average score (13.916666666666666)
Win/lose count 17.0/1.0. Average score (14.076923076923077)
Win/lose count 15.0/5.0. Average score (13.785714285714286)
Win/lose count 19.5/8.0. Average score (13.633333333333333)
Win/lose count 11.5/1.0. Average score (13.4375)
Win/lose count 16.5/6.0. Average score (13.264705882352942)
Win/lose count 14.5/4.0. Average score (13.11111111111111)
Win/lose count 22.0/6.0. Av

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***